In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import seaborn as sns
import os


In [ ]:
PLOT_DIR = "Plots"
os.makedirs(PLOT_DIR, exist_ok=True)

# =====================================================
#               DATA LOADING + SCALING
# =====================================================
def load_data(file_path_x, file_path_y, test_size=0.2, val_size=0.2):
    X = np.load(file_path_x)
    y = np.load(file_path_y)

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=test_size / (test_size + val_size),
        random_state=42,
        stratify=y_temp
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, X_val, X_test, y_train, y_val, y_test


# =====================================================
#                HYPERPARAMETERS
# =====================================================
BASE_PARAMS = {
    "activation": "relu",
    "solver": "adam",
    "batch_size": 128,
    "max_iter": 1000,
    "early_stopping": True,
    "n_iter_no_change": 50,
    "random_state": 42,
    "verbose": False,
}

HPP_MAP = {
    20: {"hidden_layer_sizes": (256, 128, 64), "learning_rate_init": 0.001, "alpha": 0.1},
    18: {"hidden_layer_sizes": (128, 128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
    16: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.001, "alpha": 0.1},
    14: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
    12: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
    10: {"hidden_layer_sizes": (128, 64), "learning_rate_init": 0.002, "alpha": 0.1},
}


# =====================================================
#              TRAINING + EVALUATION LOOP
# =====================================================
results = []
cv_scores_dict = {}

for n in range(10, 22, 2):
    if n not in HPP_MAP:
        continue

    print(f"\n===== Processing kryptonite-{n} =====")
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(
        f"Datasets/kryptonite-{n}-X.npy",
        f"Datasets/kryptonite-{n}-y.npy",
        test_size=0.2, val_size=0.2
    )

    X_cv = np.vstack((X_train, X_val))
    y_cv = np.concatenate((y_train, y_val))

    hpp = BASE_PARAMS.copy()
    hpp.update(HPP_MAP[n])

    # --- Cross-validation (disable early stopping)
    cv_params = hpp.copy()
    cv_params["early_stopping"] = False

    model_cv = MLPClassifier(**cv_params)
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model_cv, X_cv, y_cv, cv=kfold, scoring="accuracy", n_jobs=-1)
    cv_scores_dict[n] = cv_scores

    print(f"Cross-validation accuracies: {cv_scores}")
    print(f"Mean CV accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

    # --- Final training
    model = MLPClassifier(**hpp)
    model.fit(X_cv, y_cv)

    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)

    acc_train = accuracy_score(y_train, y_train_pred)
    acc_val = accuracy_score(y_val, y_val_pred)
    acc_test = accuracy_score(y_test, y_test_pred)

    results.append({
        "n": n,
        "cv_mean": cv_scores.mean(),
        "cv_std": cv_scores.std(),
        "train_acc": acc_train,
        "val_acc": acc_val,
        "test_acc": acc_test,
        "model": model,
    })

    print(f"Training Accuracy: {acc_train:.4f}")
    print(f"Validation Accuracy: {acc_val:.4f}")
    print(f"✅ Test Accuracy (held-out): {acc_test:.4f}")


# =====================================================
#              RESULTS SUMMARY + PLOTS
# =====================================================
dims = np.array([r["n"] for r in results])
cv_means = np.array([r["cv_mean"] for r in results])
cv_stds = np.array([r["cv_std"] for r in results])
test_acc = np.array([r["test_acc"] for r in results])
target_acc = np.array([0.94, 0.93, 0.92, 0.91, 0.80, 0.75])[:len(dims)]

# --- Plot 1: Accuracy vs Dimensionality
plt.figure(figsize=(8,5))
plt.plot(dims, cv_means, 'o-', color='steelblue', label='Mean CV Accuracy')
plt.fill_between(dims, cv_means - cv_stds, cv_means + cv_stds,
                 color='steelblue', alpha=0.15, label='CV ±1 SD')
plt.plot(dims, test_acc, 's--', color='darkorange', label='Test Accuracy')
plt.plot(dims, target_acc, 'r:', linewidth=2, label='Target Accuracy')
plt.xlabel("Input Dimensionality (n)")
plt.ylabel("Accuracy")
plt.title("Model Performance vs Input Dimensionality")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "accuracy_vs_dim.png"), dpi=300)
plt.close()

# --- Plot 2: Test vs Target Accuracy
width = 0.35
plt.figure(figsize=(8,5))
plt.bar(dims - width/2, test_acc, width, label='Test Accuracy', color='cornflowerblue')
plt.bar(dims + width/2, target_acc, width, label='Target Accuracy', color='lightcoral')
plt.xlabel("Dataset Dimensionality (n)")
plt.ylabel("Accuracy")
plt.title("Test vs Target Accuracy per Dataset")
plt.legend()
plt.ylim(0.7, 1.0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "test_vs_target.png"), dpi=300)
plt.close()

# --- Plot 3: Boxplot of CV scores
data = [(n, s) for n, scores in cv_scores_dict.items() for s in scores]
df = pd.DataFrame(data, columns=['Dim', 'Accuracy'])
plt.figure(figsize=(8,5))
sns.boxplot(x='Dim', y='Accuracy', data=df, palette='Blues')
sns.stripplot(x='Dim', y='Accuracy', data=df, color='black', alpha=0.6)
plt.title("Cross-Validation Accuracy Distribution per Dataset")
plt.xlabel("Input Dimensionality (n)")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, "cv_boxplot.png"), dpi=300)
plt.close()

# --- Per-Dataset Learning Curves + Confusion Matrices
for r in results:
    n = r["n"]
    model = r["model"]
    X_train, X_val, X_test, y_train, y_val, y_test = load_data(
        f"Datasets/kryptonite-{n}-X.npy",
        f"Datasets/kryptonite-{n}-y.npy",
        test_size=0.2, val_size=0.2
    )

    # --- Combined Training Loss + Validation Accuracy (single axis, your style)
    if hasattr(model, "loss_curve_"):
        plt.figure(figsize=(8, 5))

        # X-axis for training loss
        epochs = np.arange(1, len(model.loss_curve_) + 1)
        plt.plot(epochs, model.loss_curve_, color="royalblue", linewidth=2, label="Training Loss")

        # X-axis for validation accuracy (if available)
        if hasattr(model, "validation_scores_"):
            val_epochs = np.arange(1, len(model.validation_scores_) + 1)
            plt.plot(val_epochs, model.validation_scores_,
                     color="darkorange", linewidth=2, label="Validation Accuracy")

        plt.xlabel("Epoch", fontsize=11)
        plt.ylabel("Loss / Accuracy", fontsize=11)
        plt.legend(frameon=True, loc="best")
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(PLOT_DIR, f"learning_curve_kryptonite_{n}.png"), dpi=300)
        plt.close()

    # --- Confusion Matrix
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred, normalize='false')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format=".2f")
    plt.title(f"Confusion Matrix – kryptonite-{n}", fontsize=12, weight="bold")
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"cm_kryptonite_{n}.png"), dpi=300)
    plt.close()

print(f"\n✅ All training, evaluation, and plots saved in '{PLOT_DIR}/'")


===== Processing kryptonite-10 =====
Cross-validation accuracies: [0.9553125 0.964375  0.95625   0.9609375 0.96125  ]
Mean CV accuracy: 0.9596 ± 0.0034
Training Accuracy: 0.9603
Validation Accuracy: 0.9625
✅ Test Accuracy (held-out): 0.9613

===== Processing kryptonite-12 =====
Cross-validation accuracies: [0.959375   0.95677083 0.95885417 0.95989583 0.95390625]
Mean CV accuracy: 0.9578 ± 0.0022
Training Accuracy: 0.9620
Validation Accuracy: 0.9581
✅ Test Accuracy (held-out): 0.9556

===== Processing kryptonite-14 =====


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Cross-validation accuracies: [0.95245536 0.95625    0.95580357 0.95580357 0.95357143]
Mean CV accuracy: 0.9548 ± 0.0015
Training Accuracy: 0.9669
Validation Accuracy: 0.9650
✅ Test Accuracy (held-out): 0.9650

===== Processing kryptonite-16 =====


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Cross-validation accuracies: [0.93164062 0.92363281 0.921875   0.93828125 0.93300781]
Mean CV accuracy: 0.9297 ± 0.0061
Training Accuracy: 0.9570
Validation Accuracy: 0.9594
✅ Test Accuracy (held-out): 0.9428

===== Processing kryptonite-18 =====


...

Cross-validation accuracies: [0.921875   0.94288194 0.94079861 0.88385417 0.91215278]
Mean CV accuracy: 0.9203 ± 0.0216
Training Accuracy: 0.9669
Validation Accuracy: 0.9675
✅ Test Accuracy (held-out): 0.9472

===== Processing kryptonite-20 =====


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Cross-validation accuracies: [0.82203125 0.85609375 0.8421875  0.92609375 0.8878125 ]
Mean CV accuracy: 0.8668 ± 0.0366
Training Accuracy: 0.9720
Validation Accuracy: 0.9688
✅ Test Accuracy (held-out): 0.9505


/var/folders/zz/ld3qyyd543qbt84p920z_c_m0000gn/T/ipykernel_12655/4143415503.py:159: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Dim', y='Accuracy', data=df, palette='Blues')



✅ All training, evaluation, and plots saved in 'Plots/'
